In [1]:

import pandas as pd
import numpy as np
import datetime
from sklearn.ensemble import RandomForestClassifier
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn import neighbors
from sklearn.naive_bayes import GaussianNB
#import xgboost
#from xgboost import XGBClassifier
from sklearn.preprocessing import normalize
from sklearn import metrics
from ggplot import *
import matplotlib.pyplot as plt
import time

In [2]:
#loads the datasets
DF_fxrates = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/fxrates.csv")
DF_equity = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/equity.csv")
DF_10yrbond = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/10yrbond.csv")
DF_fundreturns = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/fundreturns.csv")
DF_benchmarks = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/benchmarks.csv")
DF_12monthrates = sqlContext.read.format('csv').options(header='true', inferSchema='true').load("/FileStore/tables/12monthrates.csv")



In [3]:

#convert to pandas
DF_fxrates = DF_fxrates.toPandas()
DF_equity = DF_equity.toPandas()
DF_10yrbond = DF_10yrbond.toPandas()
DF_fundreturns = DF_fundreturns.toPandas()
DF_benchmarks = DF_benchmarks.toPandas()
DF_12monthrates = DF_12monthrates.toPandas()

In [4]:
DF_fundreturns.columns

Out[ 5 ]: 
Index(['Fund Name', 'Asian Fund', 'Corporate Bond Fund', 'Dividend Fund',
 'Episode Growth Fund', 'European Select Fund',
 'Fund of Investment Trust Shares', 'Gilt & Fixed Interest Income Fund',
 'Global High Yield Bond Fund', 'Global Select Fund',
 'Global Themes Fund', 'Japan Fund', 'Japan Smaller Companies Fund',
 'Managed Growth Fund', 'North American Dividend Fund',
 'Pan European Select Fund',
 'Pan European Select Smaller Companies Fund', 'Recovery Fund',
 'Short Dated Corporate Bond Fund', 'Smaller Companies Fund',
 'UK Income Distribution Fund', 'UK Select Fund'],
 dtype='object')

In [5]:
#selects the fund analysed and its benchmark
DF_fund_analysis = DF_fundreturns[['Fund Name','Global Themes Fund']]
DF_benchmark_analysis = DF_benchmarks[['Period','M990100.N']]
DF_fund_analysis = DF_fund_analysis.rename(columns={'Fund Name': 'Period'}) 

In [6]:
#merges the two
DF_fund_analysis = DF_fund_analysis.merge(DF_benchmark_analysis, on='Period', how='inner')

#computing outperfomance
DF_fund_analysis["outperfomance"] = DF_fund_analysis[DF_fund_analysis.columns[1]].astype(float) - DF_fund_analysis[DF_fund_analysis.columns[2]].astype(float)

In [7]:
#create lags

DF_fund_analysis = pd.concat([DF_fund_analysis, DF_fund_analysis["outperfomance"].shift(), DF_fund_analysis["outperfomance"].shift(2), DF_fund_analysis["outperfomance"].shift(3), DF_fund_analysis["outperfomance"].shift(4)], axis=1)

DF_fund_analysis.columns = ['Period','Fund','Benchmark', 'outperfomance', 'outperfomanceL1', 'outperfomanceL2', 'outperfomanceL3', 'outperfomanceL4']



In [8]:
#data transformations
DF_fund_analysis["Fund_outperforms"] = np.where(DF_fund_analysis["outperfomance"]>0, 1, 0)

#tidying up the dataset
DF_fund_analysis = DF_fund_analysis.dropna()

In [9]:

# Split dataset into train and test
DF_fund_analysis['Period'] = [datetime.datetime.strptime(date,'%d/%m/%Y') for date in DF_fund_analysis['Period']]
train_df = DF_fund_analysis[DF_fund_analysis["Period"]<='2011-01-15']
train_df = train_df.reset_index()
test_df = DF_fund_analysis[DF_fund_analysis["Period"]>'2011-01-15']
test_df = test_df.reset_index()

In [10]:
# Separate the X & Y
X_cols = ['outperfomanceL1', 'outperfomanceL2', 'outperfomanceL3',
       'outperfomanceL4']
Y_cols = ['Fund_outperforms']
X_train = train_df[X_cols]
Y_train = train_df[Y_cols]
X_test = test_df[X_cols]
Y_test = test_df[Y_cols]


In [11]:
######################################################
# Random Forest Model
######################################################
forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(X_train, Y_train["Fund_outperforms"])
Y_pred = forest.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)

In [12]:
importances = forest.feature_importances_
importances

Out[ 13 ]: array([ 0.25700939, 0.24159978, 0.25132309, 0.25006774])

In [13]:
accuracy

Out[ 14 ]: 0.375

In [14]:
test_df["model_prediction"] = Y_pred
test_df["model_return"] = np.where(test_df["model_prediction"] == 1, test_df["Fund"], test_df["Benchmark"])
test_df.head()

Out[ 15 ]: 
 index Period Fund Benchmark outperfomance outperfomanceL1 \
0 338 2011-01-31 -0.054965106 -0.000455 -0.054510 0.034428 
1 339 2011-02-28 0.021039432 0.019263 0.001776 -0.054510 
2 340 2011-03-31 0.040665939 0.004685 0.035981 0.001776 
3 341 2011-04-30 0.002041887 0.001857 0.000185 0.035981 
4 342 2011-05-31 -0.010235367 -0.007657 -0.002579 0.000185 

 outperfomanceL2 outperfomanceL3 outperfomanceL4 Fund_outperforms \
0 0.005590 0.014440 0.026543 0 
1 0.034428 0.005590 0.014440 1 
2 -0.054510 0.034428 0.005590 1 
3 0.001776 -0.054510 0.034428 1 
4 0.035981 0.001776 -0.054510 0 

 model_prediction model_return 
0 1 -0.054965106 
1 0 0.0192633 
2 1 0.040665939 
3 1 0.002041887 
4 1 -0.010235367

In [15]:
test_df["fund_cumulative_return"] = (1 + test_df["Fund"].astype(float)) 
test_df["Benchmark_cumulative_return"] = (1 + test_df["Benchmark"].astype(float))  
test_df["Model_cumulative_return"] = (1 + test_df["model_return"].astype(float))  

for months in range(1,test_df.shape[0]):
  test_df["fund_cumulative_return"][months] = (1 + test_df["Fund"].astype(float)[months]) * test_df["fund_cumulative_return"][months-1]
  test_df["Benchmark_cumulative_return"][months] = (1 + test_df["Benchmark"].astype(float)[months]) * test_df["Benchmark_cumulative_return"][months-1]
  test_df["Model_cumulative_return"][months] = (1 + test_df["model_return"].astype(float)[months]) * test_df["Model_cumulative_return"][months-1] 
  
  print (test_df["Fund"][months])

/local_disk0/tmp/1548867539297-0/PythonShell.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import os
/local_disk0/tmp/1548867539297-0/PythonShell.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import pickle
/local_disk0/tmp/1548867539297-0/PythonShell.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import signal
0.021039432
0.040665939
0.002041887
-0.010235367
-0.011228952
-0.004259871
-0.054751947
-0.086986524
0.103109856
-0.060343177
-0.000621917
0.063163881
0.02942809
-0.030243616
-0.027860898
-0.075724803
0.010035778
0.000356451
0.010923546
0.049037858
0.002887473
-0.008145403
-0.006322834
0.062185963
0.038343451
0.005971587
-0.02102542
0.013770306
-0.049258613
0.029395947
-0.017289646
-0.007121793
0.009183302
-0.025697496
-0.002537862
-0.033385181
0.025671903
0.009288601
-0.008389866
0.029441688
-0.018494988
-0.008764179
0.02333247
-0.028249904
-0.001250821
0.026540176
-0.014335096
0.027663744
0.021338526
0.016175632
0.00539631
0.009570805
-0.071095782
-0.015775168
-0.054065884
-0.041688242
0.064758292
0.012179058
0.003786828
-0.038044229
0.063985153
0.040108812
0.01913632
-0.012821643
0.074476107
0.070385077
0.020921044
0.017272704
0.023774516
-0.019465609
0.02734931
0.005501454
0.019354012
0.03166258
-0.017710642
0.033531857
-0.01518653
0.015938805
0.023563115
-0.013942822
0.043653359
-0.004385653
0.021402726
-0.01115973
-0.024468816
-0.049032914
0.045699807
0.027419955
-0.006315087
0.031983415
-0.001420071
-0.00921237
-0.057006427
0.023885539
-0.06852017

In [16]:
from pyspark import SparkContext
test_df["model_return"] = test_df["model_return"].astype(float)
spark_df = spark.createDataFrame(test_df)
display(spark_df)

index,Period,Fund,Benchmark,outperfomance,outperfomanceL1,outperfomanceL2,outperfomanceL3,outperfomanceL4,Fund_outperforms,model_prediction,model_return,fund_cumulative_return,Benchmark_cumulative_return,Model_cumulative_return
338,2011-01-31T00:00:00.000+0000,-0.054965106,-4.55172E-4,-0.054509934,0.034427666999999995,0.005590444000000001,0.014439555999999996,0.026542814000000012,0,1,-0.054965106,0.945034894,0.999544828,0.945034894
339,2011-02-28T00:00:00.000+0000,0.021039432,0.019263272,0.0017761599999999989,-0.054509934,0.034427666999999995,0.005590444000000001,0.014439555999999996,1,0,0.019263272,0.9649178913899402,1.0187993318979573,0.9632393582126132
340,2011-03-31T00:00:00.000+0000,0.040665939,0.004684993,0.035980946,0.0017761599999999989,-0.054509934,0.034427666999999995,0.005590444000000001,1,1,0.040665939,1.0041571835012122,1.023572399636304,1.0024103911960864
341,2011-04-30T00:00:00.000+0000,0.002041887,0.001856938,1.84949E-4,0.035980946,0.0017761599999999989,-0.054509934,0.034427666999999995,1,1,0.002041887,1.00620755900016,1.0254731101209398,1.0044571999425347
342,2011-05-31T00:00:00.000+0000,-0.010235367,-0.007656539,-0.002578828,1.84949E-4,0.035980946,0.0017761599999999989,-0.054509934,0,1,-0.010235367,0.9959086553556191,1.0176215352598474,0.9941762118653305
343,2011-06-30T00:00:00.000+0000,-0.011228952,0.009008019,-0.020236971,-0.002578828,1.84949E-4,0.035980946,0.0017761599999999989,0,1,-0.011228952,0.9847256448682463,1.0267882893842772,0.9830126549027529
344,2011-07-31T00:00:00.000+0000,-0.004259871,-0.039695752,0.035435881,-0.020236971,-0.002578828,1.84949E-4,0.035980946,1,1,-0.004259871,0.9805308406507157,0.9860291560923747,0.9788251478014997
345,2011-08-31T00:00:00.000+0000,-0.054751947,-0.062892945,0.008140998000000003,0.035435881,-0.020236971,-0.002578828,1.84949E-4,1,1,-0.054751947,0.9268448680315423,0.9240148786098606,0.9252325651868049
346,2011-09-30T00:00:00.000+0000,-0.086986524,-0.045052535,-0.041933989,0.008140998000000003,0.035435881,-0.020236971,-0.002578828,0,1,-0.086986524,0.8462218546742397,0.8823856659507692,0.8447498004496014
347,2011-10-31T00:00:00.000+0000,0.103109856,0.064948678,0.038161178000000004,-0.041933989,0.008140998000000003,0.035435881,-0.020236971,1,0,0.064948678,0.9334756682537536,0.9396954484404212,0.8996151832295667
